# This notebook extracts statewise list of clinic information only
(the sites need to be further scraped for further analysis)

1. CSV files were created in batches, each 30 mins max approx., Eg: AL-IL, IN-NJ, NM-VT, WA-WY, NH
2. Set path to directory for csv files
3. State-wise CSVs do not have solely respective state clinics, it also includes other suggested state clinics by the site, dupilcates are removed when merging all states
4. Code needed to be modified for the state of New Hampshire (Limit to one page)

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
WAIT = 30
START = "https://www.abortionfinder.org" 

def get_HPC_base(driver, actions, state):
    """
    Navigates web browser to an abortion finder search and
    enters each CPC zip code to search for nearby abortion healthcare providers (HPCs).

    When available, the crawler collects distance, address, and url (base URL) for the
    top search result (ordered by distance in miles, alphabetical otherwise).

    Returns: HPC_data, a dictionary containing HPC names and websites
    """
    time.sleep(WAIT)

    # Enter state
    driver.find_element(By.XPATH, '//div[@class="location-box-container"]/input[@id="location-box"]').send_keys(state)

    # Tab checkbox and fill
    driver.find_element(By.XPATH, '//div[@class="search-header"]')
    actions.send_keys(Keys.TAB).send_keys(Keys.TAB).send_keys(Keys.TAB).send_keys(Keys.RETURN)
    actions.perform()
    time.sleep(WAIT)

    # Submit search
    try:
        first = driver.find_element(By.CLASS_NAME, "search-header")
    except:
        time.sleep(WAIT)
        first = driver.find_element(By.CLASS_NAME, "search-container")
    first.find_element(By.CLASS_NAME, "SubmitButton").click()
    time.sleep(WAIT)

    # Limit to physical locations
    driver.find_element(By.XPATH, '//button[@class="telehealth-service ribbon-item desktop-only  "]').click()
    time.sleep(WAIT//WAIT)
    driver.find_element(By.XPATH, '//input[@id="physical_only"]').click()
    time.sleep(WAIT//WAIT)
    driver.find_element(By.XPATH, '//button[@class="apply "]').click()

    # Find the buttons within the div element
    pages = driver.find_elements(By.XPATH, '//div[@class="PageNumberBar"]/button[@type="button"]')

    HPC_data = []
    for page in pages:
        page.click()
        
        #Get HPC names and websites 
        containers = driver.find_elements(By.XPATH, "//div[@class='ClinicCard ']/div[@class='clinic-summary-container']")
        for container in containers:
            HPC = {}
            HPC["Clinic"] = container.find_element_by_tag_name("h3").text
            HPC["Website"] = container.find_element_by_tag_name("a").get_attribute('href')
            HPC["Address"] = container.find_element_by_tag_name("p").text 
            HPC_data.append(HPC) 

    return HPC_data

In [ ]:

for state_abbv, state in STATES.items(): 
    driver = webdriver.Chrome()
    actions = ActionChains(driver)
    driver.get(START)   
    clinic = get_HPC_base(driver, actions, state)
    df = pd.DataFrame(clinic)
    df.to_csv(f'/Users/dematherese/Desktop/NEOPhilanthropy /WD/SummerProject_2023/AbortionFinder/HPC_AbortionFinder_{state_abbv}.csv')
    driver.close()
    print(state)

In [ ]:
driver.quit()

----------------------------------

In [ ]:

STATES = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NJ': 'New Jersey',
    'NH': 'New Hampshire',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}